# Geschwindigkeiten

In [ ]:
import networkx as nx
import osmnx as ox
from configobj import ConfigObj
from helpers import configreader as cfr
from helpers import osmhelper as oh

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_max_speed(cfg, edge, speeds={}):
    # example {'osmid': 26524012, 'name': 'Gailnauer Straße',
    #  'highway': 'unclassified', 'oneway': False,
    #  'length': 31.622999999999998,
    # 'geometry': <shapely.geometry.linestring.LineString object at 0x000001E6D0DC1100>}

    filters = cfr.get_filters(cfg)
    speed = 0
    max_speeds = []
    for filter_name, filter_values in filters.items():
        allowed_speed = speeds.get(filter_name.lower(), 0)
        if allowed_speed <= 0:
            continue
        for tag, allowed_values in filter_values.items():
            if tag in edge:
                if edge[tag] in allowed_values:
                    max_speeds.append(allowed_speed)

    if speeds["overrides"]:
        for override_name, allowed_speed in speeds["overrides"].items():
            filter = cfr.get_override_filter(cfg, override_name.capitalize())
            for key, value in filter.items():
                # print(key, value)
                if key in edge:
                    if edge[key] in value:
                        speed = allowed_speed

    if speed == 0 and max_speeds:
        speed = max(max_speeds)
        # print(max_speeds)
    return speed

In [ ]:
def set_speeds(G_nx, cfg=None, scenario_name=None):
    speeds = cfr.get_speeds(cfg, scenario_name)
    for u, v, k, edge in G_nx.edges(keys=True, data=True):
        max_speed = get_max_speed(cfg, edge, speeds)
        edge['length'] = max_speed
        # if max_speed == 0:
        #     edge['length'] = -1
        # else:
        #     edge['length'] = edge['length'] / (max_speed / 3.6)

    return G_nx

In [ ]:
def analyse_scenario(city, cfg, scenario, override=False, with_speed=True, folder=None):

    simplify = cfr.get_simplify(cfg)

    # load city
    G_nx = oh.get_place(placename=city, simplify=simplify,
                        clean_periphery=True,
                        file_identifier="", retain_all=True)

    # filter_graph by scenario
    # filter_scenario = oq.get_scenario_filter(cfg, scenario["name"])

    H_nx = G_nx.copy()
    # H_nx = oh.filter_graph_by_dict(H_nx, filter_scenario, drop_nodes=False)

    H_nx = set_speeds(H_nx, cfg=cfg, scenario_name=scenario["name"])

    return H_nx

In [ ]:
cfg = ConfigObj("notebook.ini")
scenarios = cfr.get_scenarios(cfg)
graphs = []
city = "Freiberg, Sachsen"
for scenario in scenarios:
    print(scenario.name)
    graph = analyse_scenario(city, cfg, scenario, override=True)
    graphs.append(graph)

    print(len(list(nx.get_edge_attributes(graph, "length").values())))

In [ ]:
for graph, scenario in zip(graphs, scenarios):
    print(scenario["name"])
    x = ox.plot_graph(graph, figsize=(
        12, 12), edge_color=ox.plot.get_edge_colors_by_attr(graph, "length"), node_size=0)